### Feed Water Composition

The following table presents the feed water analysis used for the ion exchange system design:

In [ ]:
# Display water composition table
feed_ions = ion_tracking.get('feed_mg_l', {})

# Create water quality dataframe
water_quality = []
for ion, conc in feed_ions.items():
    water_quality.append({
        'Ion': ion,
        'Concentration (mg/L)': f"{conc:.2f}",
        'Concentration (meq/L)': f"{conc/20:.3f}"  # Simplified - should use actual equiv weights
    })

df_water = pd.DataFrame(water_quality)
display(df_water.style.set_caption('Feed Water Composition'))

# Calculate total hardness
ca_mg_l = feed_ions.get('Ca_2+', 0)
mg_mg_l = feed_ions.get('Mg_2+', 0)
total_hardness = ca_mg_l * 2.5 + mg_mg_l * 4.1  # as CaCO3

print(f"\nTotal Hardness: {total_hardness:.1f} mg/L as CaCO3")
print(f"Target Effluent Hardness: {performance.get('effluent_hardness_mg_l_caco3', 5):.1f} mg/L as CaCO3")

### Design Targets

The ion exchange system is designed to meet the following performance targets:

In [ ]:
# Display design targets
targets = [
    ['Effluent Hardness', f"{performance.get('effluent_hardness_mg_l_caco3', 5):.1f} mg/L as CaCO3"],
    ['Service Flow Rate', f"{design.get('flow_m3_hr', 100):.1f} m³/hr"],
    ['Minimum Service Run', f"{performance.get('service_hours', 24):.1f} hours"],
    ['Resin Type', resin_info['display_name']]
]

# Only add alkalinity if it exists and is not None
alk_value = performance.get('effluent_alkalinity_mg_l_caco3')
if alk_value is not None:
    targets.append(['Effluent Alkalinity', f"{alk_value:.1f} mg/L as CaCO3"])

df_targets = pd.DataFrame(targets, columns=['Parameter', 'Value'])
display(df_targets.style.set_caption('Design Performance Targets'))